# FlavorSaurus

## Installing required libraries

In [ ]:
!pip install --upgrade langchain-nvidia-ai-endpoints

In [ ]:
!pip install pinecone-client

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install gradio

## Import statements

In [ ]:
import getpass
import os
import json
import time
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import gradio as gr


## Setting the API keys and environment variables

In [ ]:
with open('config.json', 'r') as file:
    config = json.load(file)

NVIDIA_API_KEY = config['API_KEYS']['NVIDIA_API_KEY']
PINECONE_API_KEY = config['API_KEYS']['PINECONE_API_KEY']
PINECONE_INDEX = config['PINECONE_INDEX']
LLM_MODEL = config['LLM_MODEL']
EMBEDDING_MODEL = config['EMBEDDING_MODEL']

# Setting the NVIDIA API KEY
os.environ["NVIDIA_API_KEY"] = NVIDIA_API_KEY
if os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    print("Valid NVIDIA_API_KEY already in environment. Delete to reset")
else:
    nvapi_key = getpass.getpass("NVAPI Key (starts with nvapi-): ")
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key

# Setting the PINECONE API KEY and PINECONE INDEX
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = PINECONE_INDEX
index = pc.Index(index_name)

# Loading the embedding model all-MiniLM-L6-v2
embedding_model = SentenceTransformer(EMBEDDING_MODEL)

## Query functions for the AI agent

In [ ]:
# Function to query Pinecone for similar recipes
def query_pinecone_for_recipes(user_query):
    # Generate the query vector from the user's input
    query_vector = embedding_model.encode(user_query).tolist()
    
    # Query Pinecone for the top 5 similar vectors
    response = index.query(vector=query_vector, top_k=5, include_metadata=True)
    return response['matches']

# Function to suggest recipes based on user query
def suggest_recipes(user_query):
    # Retrieve similar recipes from Pinecone
    matches = query_pinecone_for_recipes(user_query)
    
    # Generate recipe suggestions using NVIDIA LLM
    matching_recipes = []
    for match in matches:
        recipe_data = match['metadata']
        recipe_prompt = f"Recipe suggestion based on the query: '{user_query}':\n"
        recipe_prompt += f"Title: {recipe_data['title']}\n"
        recipe_prompt += f"Ingredients: {recipe_data['ingredients']}\n"
        recipe_prompt += f"Instructions: {recipe_data['instructions']}\n"
        
        matching_recipes.append(recipe_prompt)
    
    return matching_recipes

## Code for the AI agent

In [ ]:
template = """No need to greet everytime you reply. No need to introduce yourself everytime you reply. You are a AI Cooking Bot named 
FlavourSaurus who will give a detailed reciepe as per user's ingredients, allergies and food preferences based on {input} while taking help 
of retrived context {context} and remembering the chat history {history}. Add some dinosaur puns to the recipes. If the user's query 
is not relevant to the retrived context, come up with your own recipe."""

prompt = ChatPromptTemplate.from_template(template)

#LLM model used is meta/llama3-8b-instruct
chain = prompt | ChatNVIDIA(model=LLM_MODEL) | StrOutputParser()

# Query function used by the gradio UI
def query_llm(message, history):
    recipes = suggest_recipes(message)

    context = "\n".join([f"{i+1}. {text}" for i, text in enumerate(recipes)])

    partial_message = ""

    for chunk in chain.stream({"input":message, "context": context, "history": history}):
        partial_message += chunk
        time.sleep(0.25)
        yield partial_message

## Code for the Gradio UI

In [ ]:
default_message = """ROAR! Ah, nice to meet you! I'm FlavourSaurus, a dino-mic cooking AI bot here to help you concoct some pre-histick-tastic recipes! When you're cooking up a storm, you can count on me to assist with ingredient suggestions, allergy-friendly options, and palate-pleasing dishes that cater to your food preferences. So, what's on your plate?"""

# Gradio Interface
gradio_interface = gr.ChatInterface(
        query_llm,
        chatbot=gr.Chatbot(value=[[None, default_message]]),
        textbox=gr.Textbox(placeholder="Ask FlavourSaurus for food recipies as per your ingredients, allergies and food preferences!", container=False, scale=7),
        title="FlavourSaurus, a dino-mic cooking AI bot",
        description=f"FlavourSaurus is a cooking AI bot who will suggest you food recipies as per your ingredients, allergies and food preferences",
        theme='abidlabs/Lime', # themes at https://huggingface.co/spaces/gradio/theme-gallery
        retry_btn=None,
        undo_btn="Delete Previous",
        clear_btn="Clear",

)

## Launching the AI agent with Gradio UI interface

In [ ]:
#Following code will launch the UI on localhost (For example: Running on local URL:  http://127.0.0.1:7860)
gradio_interface.launch()

## Closing the Gradio UI Interface

In [ ]:
gradio_interface.close()